In [1]:
# Notebook para convertir notebooks a PDF/HTML con mejor distribución
import os
import nbformat
from nbconvert import PDFExporter, HTMLExporter
from nbconvert.writers import FilesWriter
import ipywidgets as widgets
from IPython.display import display, clear_output
from traitlets.config import Config

# ---------- Funciones ----------

def find_notebooks_recursively(root_dir='.'):
    """Busca todos los notebooks .ipynb en root_dir y subdirectorios."""
    notebooks = []
    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if f.endswith('.ipynb'):
                rel_path = os.path.relpath(os.path.join(dirpath, f), root_dir)
                notebooks.append(rel_path)
    return notebooks

def convert_notebook(notebook_path, to_pdf=True):
    """Convierte un notebook a PDF o HTML con mejor distribución."""
    # Cargar notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    if to_pdf:
        # Configuración LaTeX para PDF
        c = Config()
        c.LatexExporter.section_numbering = False  # Sin numeración de títulos
        c.LatexExporter.template_file = 'article'  # Base template
        c.PDFExporter.exclude_input = False
        c.PDFExporter.exclude_output = False

        # Mejor distribución
        c.LatexExporter.template_extensions = ['.tplx']
        pdf_exporter = PDFExporter(config=c)

        # Inyectar paquetes LaTeX para mejor presentación
        pdf_exporter.template_file = 'article'
        pdf_exporter.template_extensions = ['.tplx']
        pdf_exporter.extra_template_basedirs = ['.']
        pdf_exporter.preprocessors = []

        # Inyectar paquetes directamente en LaTeX header
        pdf_exporter.template_file = None  # No template por defecto
        pdf_exporter.template_name = None
        pdf_exporter.template_paths = []
        pdf_exporter.template = r"""
((*- extends 'article.tplx' -*))

((* block docclass *))
\documentclass[a4paper,12pt]{article}
((* endblock docclass *))

((* block header *))
\usepackage[a4paper,margin=1in]{geometry}
\usepackage{graphicx}
\usepackage{fvextra} % Código con saltos de línea
\DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines=true}
\usepackage{needspace} % Evitar cortes de celdas
\usepackage{float} % Para gráficos/tablas
((* endblock header *))
"""
    else:
        pdf_exporter = HTMLExporter()
        pdf_exporter.exclude_input = False
        pdf_exporter.exclude_output = False

    # Convertir notebook
    try:
        body, resources = pdf_exporter.from_notebook_node(notebook)
    except Exception as e:
        print(f"Error convirtiendo a {'PDF' if to_pdf else 'HTML'}: {e}")
        if to_pdf:
            print("Intentando exportar a HTML en su lugar...")
            pdf_exporter = HTMLExporter()
            body, resources = pdf_exporter.from_notebook_node(notebook)
            to_pdf = False
        else:
            return

    # Guardar archivo
    output_path = notebook_path.rsplit('.', 1)[0]
    writer = FilesWriter()
    writer.write(body, resources, notebook_name=output_path)
    print(f"✅ Notebook '{notebook_path}' convertido a {'PDF' if to_pdf else 'HTML'} → {output_path}.{ 'pdf' if to_pdf else 'html'}")

# ---------- Widget interactivo ----------

notebooks = find_notebooks_recursively('.')

if not notebooks:
    print("No hay notebooks en el directorio actual ni en subdirectorios.")
else:
    dropdown = widgets.Dropdown(
        options=notebooks,
        description='Notebook:',
        layout=widgets.Layout(width='600px')
    )

    pdf_checkbox = widgets.Checkbox(
        value=True,
        description='Exportar a PDF (si falla, HTML)',
        indent=False
    )

    button = widgets.Button(description="Convertir", button_style='success')
    output = widgets.Output()

    def on_button_click(b):
        with output:
            clear_output()
            notebook_path = dropdown.value
            convert_notebook(notebook_path, to_pdf=pdf_checkbox.value)

    button.on_click(on_button_click)

    display(dropdown, pdf_checkbox, button, output)


Dropdown(description='Notebook:', layout=Layout(width='600px'), options=('work/notebooks/convertir_a_pdf.ipynb…

Checkbox(value=True, description='Exportar a PDF (si falla, HTML)', indent=False)

Button(button_style='success', description='Convertir', style=ButtonStyle())

Output()